# 중요!!!!!!! 데이터 프레임 출력한거 있으면 지우고 업데이트!!

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import os

import pickle

# pd.set_option('display.max_columns', None) ## 모든 열 출력
# pd.set_option('display.max_rows', None) ## 모든 행 출력

## 경로 맞추기

In [2]:
# 현재 작업 디렉토리 가져오기
current_directory = os.getcwd()
print("현재 디렉토리:", current_directory)

# 3개 위의 폴더 찾기
three_levels_up = os.path.dirname(os.path.dirname(os.path.dirname(current_directory)))
print("3개 위의 디렉토리:", three_levels_up)

# 3개 위의 폴더 찾기
data_path = os.path.join(three_levels_up,'ETF_data')
print("데이터 경로:", data_path)

현재 디렉토리: c:\Users\howsr\workspace\DSHY_main\DSHY\EDA 실습\농협 공모전
3개 위의 디렉토리: c:\Users\howsr\workspace\DSHY_main
데이터 경로: c:\Users\howsr\workspace\DSHY_main\ETF_data


In [3]:
# 현재 작업 디렉토리 확인
print("현재 작업 디렉토리:", os.getcwd())

# 새로운 디렉토리로 변경
os.chdir(data_path)

# 변경된 작업 디렉토리 확인
print("변경된 작업 디렉토리:", os.getcwd())

현재 작업 디렉토리: c:\Users\howsr\workspace\DSHY_main\DSHY\EDA 실습\농협 공모전
변경된 작업 디렉토리: c:\Users\howsr\workspace\DSHY_main\ETF_data


# 데이터 불러오기

In [4]:
해외종목정보 = pd.read_csv("NH_CONTEST_NW_FC_STK_IEM_IFO.csv", encoding="EUC-KR", skipinitialspace=True)
종목일자별시세 = pd.read_csv("NH_CONTEST_STK_DT_QUT.csv", encoding="EUC-KR", skipinitialspace=True)
주식일별정보 = pd.read_csv("NH_CONTEST_NHDATA_STK_DD_IFO.csv", encoding="EUC-KR", skipinitialspace=True)
매수매도계좌정보 = pd.read_csv("NH_CONTEST_NHDATA_IFW_OFW_IFO.csv", encoding="EUC-KR", skipinitialspace=True)
고객보유정보 = pd.read_csv("NH_CONTEST_NHDATA_CUS_TP_IFO.csv", encoding="EUC-KR", skipinitialspace=True)
ETF구성종목정보 = pd.read_csv("NH_CONTEST_DATA_ETF_HOLDINGS.csv", encoding="EUC-KR", skipinitialspace=True)
ETF배당내역 = pd.read_csv("NH_CONTEST_DATA_HISTORICAL_DIVIDEND.csv", encoding="EUC-KR", skipinitialspace=True)
ETF점수정보 = pd.read_csv("NH_CONTEST_ETF_SOR_IFO.csv", encoding="EUC-KR", skipinitialspace=True)

data_name = ["해외종목정보", "종목일자별시세", "주식일별정보", "매수매도계좌정보", "고객보유정보", "ETF구성종목정보", "ETF배당내역", "ETF점수정보"]

In [5]:
# 변수명 변경 + 공백제거
해외종목정보.columns = ["티커종목코드", "외화증권한글명", "외화증권영문명", "주식/ETF구분코드",
                   "상장주식총수량", "외화시장구분코드", "회사주소", "웹주소", "업종분류명",
                   "CEO명", "영문사업개요내용", "섹터분류명", "산업명", "시가총액"]
해외종목정보 = 해외종목정보.map(lambda x: x.strip() if isinstance(x, str) else x)

종목일자별시세.columns = ["거래일자", "티커종목코드", "종목시가", "종목고가", "종목저가",
                   "종목종가", "전일대비증감가격", "전일대비증감율", "누적거래수량",
                   "거래대금", "매도체결합계수량", "매수체결합계수량", "환율"]
종목일자별시세 = 종목일자별시세.map(lambda x: x.strip() if isinstance(x, str) else x)

주식일별정보.columns = ["거래일자", "티커종목코드", "총보유계좌수", "총보유수량",
                   "당사평균보유수량", "당사평균보유비중비율", "당사평균평가손익",
                   "당사평균매입단가", "당사평균수익율", "당사평균보유기간일수",
                   "분포상위10퍼센트수치", "분포상위30퍼센트수치", "분포상위50퍼센트수치",
                   "분포상위70퍼센트수치", "분포상위90퍼센트수치", "기준종가", "손실투자자비율",
                   "수익투자자비율", "신규매수계좌수", "전량매도계좌수", "종목조회건수",
                   "관심종목등록건수"]
주식일별정보 = 주식일별정보.map(lambda x: x.strip() if isinstance(x, str) else x)

매수매도계좌정보.columns = ["기준일자", "티커종목코드", "유입/유출구분코드", "유입유출티커코드",
                    "유입유출금액비중", "유입유출랭크"]
매수매도계좌정보 = 매수매도계좌정보.map(lambda x: x.strip() if isinstance(x, str) else x)

고객보유정보.columns = ["기준일자", "티커종목코드", "고객구성대분류코드", "고객구성중분류코드",
                  "고객구성계좌수비율", "고객구성투자비율"]
고객보유정보 = 고객보유정보.map(lambda x: x.strip() if isinstance(x, str) else x)

ETF구성종목정보.columns = ["대상ETF티커", "ETF개별구성종목티커", "보유종목의가치(USD)", "보유종목의영문명", "보유종목의한글명",
                     "보유종목의주수(주)", "보유종목의비중(%)", "보유종목의타입(ST:주식,EF:ETF,EN:ETN,SSEF:Single-StockETF)"]
ETF구성종목정보 = ETF구성종목정보.map(lambda x: x.strip() if isinstance(x, str) else x)

ETF배당내역.columns = ["대상ETF티커", "배당락일(YYYYMMDD)", "배당금", "수정배당금", "배당기준일(YYYYMMDD)",
                   "지급일(YYYYMMDD)", "공시일(YYYYMMDD)", "배당주기(Quarterly:분기배당,Weekly:주배당,Monthly:월배당,SemiAnnual:반기배당,Annual:연배당,Other:알수없음)"]
ETF배당내역 = ETF배당내역.map(lambda x: x.strip() if isinstance(x, str) else x)

ETF점수정보.columns = ["거래일자", "대상ETF티커", "1개월총수익율", "3개월총수익율",
                   "1년총수익율", "ETF점수", "ETFZ점수", "Z점수순위",
                   "누적수익율Z점수", "정보비율Z점수", "샤프지수Z점수", "상관관계Z점수",
                   "트래킹에러Z점수", "최대낙폭Z점수", "변동성Z점수"]
ETF점수정보 = ETF점수정보.map(lambda x: x.strip() if isinstance(x, str) else x)

# EDA

In [6]:
print(해외종목정보["주식/ETF구분코드"].value_counts())
print((해외종목정보["티커종목코드"] == "AAPB").sum()) # AAPL 2배 레버리지 SSEF
해외종목정보[해외종목정보["주식/ETF구분코드"] == "ETF"]["티커종목코드"].unique().shape 


# ETF가 SSEF와 구분되어있지 않음
# ETF 167 종목

주식/ETF구분코드
주식     2929
ETF     167
Name: count, dtype: int64
1


(167,)

In [7]:
해외종목정보["외화시장구분코드"].unique()

array(['NYSE', 'NASD', '-'], dtype=object)

In [8]:
print(ETF점수정보["대상ETF티커"].unique())
print(ETF점수정보["대상ETF티커"].unique().shape)

# ETF 55종목

['AIYY' 'CIBR' 'DIA' 'DIV' 'IHI' 'IJR' 'JEPI' 'KLIP' 'KRE' 'MOAT' 'MSOS'
 'NVDY' 'VTI' 'XBI' 'XLF' 'XLU' 'DGRO' 'DIVO' 'MGK' 'QQQ' 'SCHD' 'SPYV'
 'VIG' 'AWAY' 'DGRW' 'EETH' 'FBY' 'IBB' 'IVV' 'IWM' 'JEPQ' 'NOBL' 'RYLD'
 'SKYY' 'SOXQ' 'SOXX' 'SPY' 'SPYD' 'XLK' 'XLP' 'XLV' 'AMZY' 'APLY' 'GOOY'
 'IJH' 'ITA' 'OARK' 'QQQM' 'SMH' 'SPHD' 'SPLG' 'SPYG' 'TSLY' 'VOO' 'VYM']
(55,)


In [9]:
ETF구성종목정보["보유종목의타입(ST:주식,EF:ETF,EN:ETN,SSEF:Single-StockETF)"].unique()

# 타입이 4종류?

array(['ST', 'EF', '-', 'RT'], dtype=object)

In [10]:
print(sorted(ETF구성종목정보["대상ETF티커"].unique()))
print(ETF구성종목정보["대상ETF티커"].unique().shape)

# ETF 122개

['AAPB', 'AAPU', 'AIQ', 'AMDL', 'AMZU', 'AOR', 'ARKF', 'ARKG', 'ARKK', 'ARKQ', 'ARKW', 'ARKX', 'AWAY', 'BOTZ', 'CIBR', 'CLOU', 'CONL', 'COPX', 'CWEB', 'DFEN', 'DGRO', 'DGRW', 'DIA', 'DIV', 'DIVO', 'DPST', 'DRIV', 'FAS', 'FBL', 'GDX', 'GGLL', 'GPIQ', 'GPIX', 'GRID', 'HDRO', 'IBB', 'ICLN', 'IEMG', 'IHI', 'ITA', 'IVV', 'IWM', 'JEPI', 'JEPQ', 'JETS', 'KBWY', 'KLIP', 'KORU', 'KRBN', 'LABU', 'LIT', 'METV', 'MGK', 'MOAT', 'MSFU', 'MSOX', 'NAIL', 'NOBL', 'NUSI', 'NVD', 'NVDL', 'NVDU', 'PAVE', 'PFF', 'PGX', 'QCLN', 'QLD', 'QQQ', 'QQQM', 'QYLD', 'RETL', 'RPAR', 'RYLD', 'SCHD', 'SDIV', 'SKYY', 'SMH', 'SOXL', 'SOXQ', 'SOXX', 'SPHD', 'SPLG', 'SPXL', 'SPY', 'SPYD', 'SPYG', 'SPYV', 'SSO', 'SVOL', 'TAN', 'TECL', 'TLTW', 'TMF', 'TNA', 'TQQQ', 'TSL', 'TSLL', 'UDOW', 'UPRO', 'URA', 'URTY', 'USD', 'VEA', 'VIG', 'VNQ', 'VOO', 'VT', 'VTI', 'VWO', 'VYM', 'WEBL', 'XBI', 'XLE', 'XLF', 'XLK', 'XLP', 'XLRE', 'XLU', 'XLV', 'XYLD', 'YINN', 'YMAX']
(122,)


# 거래일자, 고객 중분류 딕셔너리 코드

In [11]:
# 거래일자, 고객 중분류 검색하면 딕셔너리 만드는 코드
# 여기에는 함수만 올림
def filter_date(df, date):
    if '기준일자' in df.columns:
        return df[df['기준일자'] == date]
    elif '거래일자' in df.columns:
        return df[df['거래일자'] == date]

def filter_code(df, ticker_code):
    if '티커종목코드' in df.columns:
        return df[df['티커종목코드'] == ticker_code]
    elif '대상ETF티커' in df.columns:
        return df[df['대상ETF티커'] == ticker_code]

def filter_bigConstraint(df, big_constraint):
    return df[df['고객구성대분류코드'] == big_constraint]

# 날짜, 고객 별로 ETF를 얼마나 갖고 있는지 저장할 dictionary
# 이름을 뭐로 해야할지 모르겠어서 name_mijung ㅋㅋ
name_mijung = {}

def add_or_concat(key, new_data):
    # 데이터프레임 생성 (new_data는 새로운 row을 담은 DataFrame)
    new_df = pd.DataFrame(new_data)

    # 해당 key에 데이터프레임이 없는 경우, 새로운 데이터프레임 추가
    if key not in name_mijung:
        name_mijung[key] = new_df
    else:
        # 이미 데이터프레임이 있다면, pd.concat으로 row 추가
        name_mijung[key] = pd.concat([name_mijung[key], new_df])

def calculate_actual_numbers(고객보유정보, 주식일별정보, date, ticker_code, big_constraint):
    filtered_stock_info = filter_date((filter_code(주식일별정보, ticker_code)), date)
    rowIndex_stock_info = filtered_stock_info.index[0]
    filtered_customer_info = filter_bigConstraint(filter_date((filter_code(고객보유정보, ticker_code)), date), big_constraint)
    for rowIndex in filtered_customer_info.index:
        accounts = filtered_customer_info.loc[rowIndex, "고객구성계좌수비율"] * filtered_stock_info.loc[rowIndex_stock_info, "총보유계좌수"]
        stocks = filtered_customer_info.loc[rowIndex, "고객구성투자비율"] * filtered_stock_info.loc[rowIndex_stock_info, "총보유수량"]
        dont_know_proper_name = pd.DataFrame({"보유고객계좌수":[accounts], "보유고객주식수":[stocks]}, index=[ticker_code])
        add_or_concat((date, filtered_customer_info.loc[rowIndex, "고객구성중분류코드"]), dont_know_proper_name)

In [12]:
# #이거 돌리면 위에서 저장한 name_mijung이라는 딕셔너리가 만들어짐
# tickerCode = list(주식일별정보['티커종목코드'].unique())
# for tC in tickerCode:
# # tC = tickerCode[189]
#     imsi = 주식일별정보[주식일별정보['티커종목코드'] == tC]
#     ETF_dates = list(imsi['거래일자'].unique())
#     for date in ETF_dates:
#         for i in range(1,4): #고객 대분류 번호가 1, 2, 3
#             calculate_actual_numbers(고객보유정보, 주식일별정보, date, tC, i)
#             print(f"{date}, {tC}, 대분류 :{i}")

# # ETF 하나 당 9.8sec, 우리가 가진 ETF 1277개
# # ETA : 3시간 42분 
# # 우리 멋쟁이 코드 화이팅 !

# # 파일 저장
# # folder_path 세팅해주세요
# file_path = f"{folder_path}날짜및고객분류별보유ETF.pkl"
# with open(file_path, 'wb') as f:
#     pickle.dump(name_mijung, f)

In [13]:
with open('날짜및고객분류별보유ETF.pkl', 'rb') as f:
    name_mijung = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: '날짜및고객분류별보유ETF.pkl'

In [16]:
name_mijung.keys()

dict_keys([(20240726, 11), (20240726, 12), (20240726, 24), (20240726, 22), (20240726, 25), (20240726, 23), (20240726, 21), (20240726, 34), (20240726, 32), (20240726, 31), (20240726, 33), (20240729, 12), (20240729, 11), (20240729, 22), (20240729, 23), (20240729, 24), (20240729, 25), (20240729, 21), (20240729, 33), (20240729, 34), (20240729, 31), (20240729, 32), (20240709, 11), (20240709, 12), (20240709, 22), (20240709, 21), (20240709, 25), (20240709, 23), (20240709, 24), (20240709, 33), (20240709, 34), (20240709, 31), (20240709, 32), (20240802, 11), (20240802, 12), (20240802, 25), (20240802, 22), (20240802, 21), (20240802, 23), (20240802, 24), (20240802, 34), (20240802, 32), (20240802, 31), (20240802, 33), (20240715, 12), (20240715, 11), (20240715, 23), (20240715, 22), (20240715, 21), (20240715, 24), (20240715, 25), (20240715, 34), (20240715, 33), (20240715, 31), (20240715, 32), (20240823, 12), (20240823, 11), (20240823, 22), (20240823, 24), (20240823, 23), (20240823, 25), (20240823, 21

In [ ]:
name_mijung[(20240726, 11)]

# ETF 테이블 만들기

## ETF 관련 EDA

In [14]:
ETF점수정보.columns

Index(['거래일자', '대상ETF티커', '1개월총수익율', '3개월총수익율', '1년총수익율', 'ETF점수', 'ETFZ점수',
       'Z점수순위', '누적수익율Z점수', '정보비율Z점수', '샤프지수Z점수', '상관관계Z점수', '트래킹에러Z점수',
       '최대낙폭Z점수', '변동성Z점수'],
      dtype='object')

In [15]:
ETF해외종목정보 = 해외종목정보[해외종목정보["주식/ETF구분코드"] == "ETF"]

In [16]:
print(주식일별정보["티커종목코드"].unique().shape)
print(ETF해외종목정보["티커종목코드"].unique().shape)
np.isin(주식일별정보["티커종목코드"].unique(), ETF해외종목정보["티커종목코드"].unique()).sum()

# 일부 존재 안함 (주식일별정보가 NH데이터 기반이니까 정보가 없는듯?)

(1277,)
(167,)


152

In [17]:
print(종목일자별시세["티커종목코드"].unique().shape)
print(ETF해외종목정보["티커종목코드"].unique().shape)
np.isin(종목일자별시세["티커종목코드"].unique(), ETF해외종목정보["티커종목코드"].unique()).sum()

# 전부 존재

(3094,)
(167,)


167

In [18]:
print(ETF해외종목정보["티커종목코드"].unique().shape)
print(ETF구성종목정보["대상ETF티커"].unique().shape)
np.isin(ETF해외종목정보["티커종목코드"].unique(), ETF구성종목정보["대상ETF티커"].unique()).sum()

# 전부 존재

(167,)
(122,)


122

In [19]:
print(ETF배당내역["대상ETF티커"].unique().shape)
print(ETF구성종목정보["대상ETF티커"].unique().shape)
np.isin(ETF배당내역["대상ETF티커"].unique(), ETF구성종목정보["대상ETF티커"].unique()).sum()

# 전부 존재하지는 않음 (새로 생겨서 배당이 없나??, 또는 없어졌나??)

(1631,)
(122,)


113

In [20]:
print(ETF배당내역["대상ETF티커"].unique().shape)
print(ETF점수정보["대상ETF티커"].unique().shape)
np.isin(ETF배당내역["대상ETF티커"].unique(), ETF점수정보["대상ETF티커"].unique()).sum()



(1631,)
(55,)


54

In [21]:
bool_array = np.isin(ETF점수정보["대상ETF티커"].unique(), ETF배당내역["대상ETF티커"].unique())
ETF점수정보["대상ETF티커"].unique()[np.where(~bool_array)[0]]

# 1개 ETF 없어진게 아니라 배당날짜가 21년이라 없는 것

array(['MSOS'], dtype=object)

In [22]:
print(ETF구성종목정보["대상ETF티커"].unique().shape)
print(ETF점수정보["대상ETF티커"].unique().shape)
np.isin(ETF구성종목정보["대상ETF티커"].unique(), ETF점수정보["대상ETF티커"].unique()).sum()

(122,)
(55,)


42

In [23]:
bool_array = np.isin(ETF점수정보["대상ETF티커"].unique(), ETF구성종목정보["대상ETF티커"].unique())
ETF점수정보["대상ETF티커"].unique()[np.where(~bool_array)[0]]

# 왜 없는지 모르겠음

array(['AIYY', 'IJR', 'KRE', 'MSOS', 'NVDY', 'EETH', 'FBY', 'AMZY',
       'APLY', 'GOOY', 'IJH', 'OARK', 'TSLY'], dtype=object)

In [24]:
ETF구성종목정보.columns

Index(['대상ETF티커', 'ETF개별구성종목티커', '보유종목의가치(USD)', '보유종목의영문명', '보유종목의한글명',
       '보유종목의주수(주)', '보유종목의비중(%)',
       '보유종목의타입(ST:주식,EF:ETF,EN:ETN,SSEF:Single-StockETF)'],
      dtype='object')

## ETF 테이블 생성

In [25]:
# ETF해외종목정보[["티커종목코드", "외화증권한글명", "외화증권영문명"]] # 167
# 종목일자별시세[["거래일자", "티커종목코드", "전일대비증감율"]] # 167 이상
# 주식일별정보[["거래일자", "티커종목코드", "당사평균매입단가", "당사평균보유기간일수", "관심종목등록건수", "종목조회건수"]] # 152 (NH니까)
# # ETF배당내역
# # ETF구성종목정보
# ETF점수정보 # 55

In [26]:
ETF해외종목정보 = 해외종목정보[해외종목정보["주식/ETF구분코드"] == "ETF"]

In [27]:
# NH에 맞추기 위해 주식일별정보를 base로 사용
etf_df = 주식일별정보.copy()
etf_df = etf_df[np.isin(etf_df["티커종목코드"], ETF해외종목정보["티커종목코드"])]
etf_df['티커종목코드'].unique().shape

(152,)

In [28]:
# 주식일별정보 + ETF해외종목정보
etf_df = pd.merge(etf_df, ETF해외종목정보[["티커종목코드", "외화증권영문명"]], on='티커종목코드', how='left')
first_cols = ["티커종목코드", "외화증권영문명", "거래일자"]
other_cols = [col for col in etf_df.columns if col not in first_cols]
etf_df = etf_df[first_cols + other_cols]
# etf_df = etf_df[["티커종목코드", "외화증권영문명", "거래일자", "당사평균매입단가", "당사평균보유기간일수", "관심종목등록건수", "종목조회건수"]]
print(etf_df.info())
etf_df = etf_df.sort_values(by=["티커종목코드", '거래일자'])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9728 entries, 0 to 9727
Data columns (total 23 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   티커종목코드       9728 non-null   object 
 1   외화증권영문명      9728 non-null   object 
 2   거래일자         9728 non-null   int64  
 3   총보유계좌수       9728 non-null   int64  
 4   총보유수량        9728 non-null   int64  
 5   당사평균보유수량     9728 non-null   float64
 6   당사평균보유비중비율   9728 non-null   float64
 7   당사평균평가손익     9728 non-null   float64
 8   당사평균매입단가     9728 non-null   float64
 9   당사평균수익율      9728 non-null   float64
 10  당사평균보유기간일수   9728 non-null   float64
 11  분포상위10퍼센트수치  9728 non-null   float64
 12  분포상위30퍼센트수치  9728 non-null   float64
 13  분포상위50퍼센트수치  9728 non-null   float64
 14  분포상위70퍼센트수치  9728 non-null   float64
 15  분포상위90퍼센트수치  9728 non-null   float64
 16  기준종가         9728 non-null   float64
 17  손실투자자비율      9728 non-null   float64
 18  수익투자자비율      9728 non-null   float64
 19  신규매수계좌

In [29]:
# + 종목일자별시세
etf_df = pd.merge(etf_df, 종목일자별시세[["거래일자", "티커종목코드", "종목종가", "전일대비증감율", "누적거래수량", "거래대금", "매도체결합계수량", "매수체결합계수량"]], on=['티커종목코드', '거래일자'], how='left')
print(etf_df.info()) # null : 470

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9728 entries, 0 to 9727
Data columns (total 29 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   티커종목코드       9728 non-null   object 
 1   외화증권영문명      9728 non-null   object 
 2   거래일자         9728 non-null   int64  
 3   총보유계좌수       9728 non-null   int64  
 4   총보유수량        9728 non-null   int64  
 5   당사평균보유수량     9728 non-null   float64
 6   당사평균보유비중비율   9728 non-null   float64
 7   당사평균평가손익     9728 non-null   float64
 8   당사평균매입단가     9728 non-null   float64
 9   당사평균수익율      9728 non-null   float64
 10  당사평균보유기간일수   9728 non-null   float64
 11  분포상위10퍼센트수치  9728 non-null   float64
 12  분포상위30퍼센트수치  9728 non-null   float64
 13  분포상위50퍼센트수치  9728 non-null   float64
 14  분포상위70퍼센트수치  9728 non-null   float64
 15  분포상위90퍼센트수치  9728 non-null   float64
 16  기준종가         9728 non-null   float64
 17  손실투자자비율      9728 non-null   float64
 18  수익투자자비율      9728 non-null   float64
 19  신규매수계좌

In [30]:
# + ETF점수정보
etf_df = pd.merge(etf_df, ETF점수정보, left_on=['티커종목코드', '거래일자'], right_on=['대상ETF티커', '거래일자'], how='left')
print(etf_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9728 entries, 0 to 9727
Data columns (total 43 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   티커종목코드       9728 non-null   object 
 1   외화증권영문명      9728 non-null   object 
 2   거래일자         9728 non-null   int64  
 3   총보유계좌수       9728 non-null   int64  
 4   총보유수량        9728 non-null   int64  
 5   당사평균보유수량     9728 non-null   float64
 6   당사평균보유비중비율   9728 non-null   float64
 7   당사평균평가손익     9728 non-null   float64
 8   당사평균매입단가     9728 non-null   float64
 9   당사평균수익율      9728 non-null   float64
 10  당사평균보유기간일수   9728 non-null   float64
 11  분포상위10퍼센트수치  9728 non-null   float64
 12  분포상위30퍼센트수치  9728 non-null   float64
 13  분포상위50퍼센트수치  9728 non-null   float64
 14  분포상위70퍼센트수치  9728 non-null   float64
 15  분포상위90퍼센트수치  9728 non-null   float64
 16  기준종가         9728 non-null   float64
 17  손실투자자비율      9728 non-null   float64
 18  수익투자자비율      9728 non-null   float64
 19  신규매수계좌

In [31]:
df = etf_df.set_index(['티커종목코드', '거래일자'])

In [32]:
df

외화증권영문명  총보유계좌수  총보유수량  당사평균보유수량  \
티커종목코드 거래일자                                                                  
AAPB   20240528      GRANITESHARES AAPL DAILY ETF     134  14026    104.67   
       20240529      GRANITESHARES AAPL DAILY ETF     139  14259    102.58   
       20240530      GRANITESHARES AAPL DAILY ETF     140  14324    102.31   
       20240531      GRANITESHARES AAPL DAILY ETF     138  14166    102.65   
       20240603      GRANITESHARES AAPL DAILY ETF     134  11343     84.65   
...                                           ...     ...    ...       ...   
YMAX   20240821  YIELDMAX UNIVRS FD OPT INCOM ETF     173  30181    174.46   
       20240822  YIELDMAX UNIVRS FD OPT INCOM ETF     174  30395    174.68   
       20240823  YIELDMAX UNIVRS FD OPT INCOM ETF     172  30281    176.05   
       20240826  YIELDMAX UNIVRS FD OPT INCOM ETF     172  29999    174.41   
       20240827  YIELDMAX UNIVRS FD OPT INCOM ETF     172  29984    174.33   

                 당사평균보유비중비율   당사평균평가손익  당사평균매입단가  당사평균수익율  당사평균보유기간일수  \
티커종목코드 거래일자                                                             
AAPB   20240528       14.70  195128.25  26719.95     6.96         1.0   
       20240529       14.53  175534.24  26845.70     6.37         1.0   
       20240530       14.54  196985.28  26832.75     7.16         1.0   
       20240531       14.37  251587.12  26842.32     9.11         1.0   
       20240603       14.21  273713.13  26786.26    12.22         3.0   
...                     ...        ...       ...      ...         ...   
YMAX   20240821       10.91 -518520.12  27134.42   -11.08         1.0   
       20240822       10.84 -486999.29  27137.28   -10.39         1.0   
       20240823       10.76 -559175.59  27147.45   -11.84         1.0   
       20240826       10.68 -508738.99  27157.81   -10.86         3.0   
       20240827       10.67 -545871.90  27123.61   -11.66         1.0   

                 분포상위10퍼센트수치  ...  ETF점수  ETFZ점수  Z점수순위  누적수익율Z점수  정보비율Z점수  \
티커종목코드 거래일자                   ...                                            
AAPB   20240528  23273.00000  ...    NaN     NaN    NaN       NaN      NaN   
       20240529  23273.00000  ...    NaN     NaN    NaN       NaN      NaN   
       20240530  23331.00000  ...    NaN     NaN    NaN       NaN      NaN   
       20240531  23273.00000  ...    NaN     NaN    NaN       NaN      NaN   
       20240603  23273.00000  ...    NaN     NaN    NaN       NaN      NaN   
...                      ...  ...    ...     ...    ...       ...      ...   
YMAX   20240821  25434.30336  ...    NaN     NaN    NaN       NaN      NaN   
       20240822  25434.30336  ...    NaN     NaN    NaN       NaN      NaN   
       20240823  25520.80000  ...    NaN     NaN    NaN       NaN      NaN   
       20240826  25547.57143  ...    NaN     NaN    NaN       NaN      NaN   
       20240827  25520.80000  ...    NaN     NaN    NaN       NaN      NaN   

                 샤프지수Z점수  상관관계Z점수  트래킹에러Z점수  최대낙폭Z점수  변동성Z점수  
티커종목코드 거래일자                                                   
AAPB   20240528      NaN      NaN       NaN      NaN     NaN  
       20240529      NaN      NaN       NaN      NaN     NaN  
       20240530      NaN      NaN       NaN      NaN     NaN  
       20240531      NaN      NaN       NaN      NaN     NaN  
       20240603      NaN      NaN       NaN      NaN     NaN  
...                  ...      ...       ...      ...     ...  
YMAX   20240821      NaN      NaN       NaN      NaN     NaN  
       20240822      NaN      NaN       NaN      NaN     NaN  
       20240823      NaN      NaN       NaN      NaN     NaN  
       20240826      NaN      NaN       NaN      NaN     NaN  
       20240827      NaN      NaN       NaN      NaN     NaN  

[9728 rows x 41 columns]

In [33]:
df.loc[('SPY', 20240528)]

외화증권영문명        SPDR S&P 500 ETF
총보유계좌수                    15908
총보유수량                    165025
당사평균보유수량                  10.37
당사평균보유비중비율                23.36
당사평균평가손익             1749776.26
당사평균매입단가              566350.26
당사평균수익율                   30.43
당사평균보유기간일수                  1.0
분포상위10퍼센트수치            465337.0
분포상위30퍼센트수치            522968.0
분포상위50퍼센트수치            547888.6
분포상위70퍼센트수치          607796.143
분포상위90퍼센트수치            695440.0
기준종가                 722897.276
손실투자자비율                     0.0
수익투자자비율                     1.0
신규매수계좌수                      18
전량매도계좌수                      27
종목조회건수                     2876
관심종목등록건수                     97
종목종가                     529.81
전일대비증감율                    0.07
누적거래수량               36269602.0
거래대금              19215997836.0
매도체결합계수량             13157567.0
매수체결합계수량             23112267.0
대상ETF티커                     SPY
1개월총수익율                  4.7546
3개월총수익율                  4.5774
1년총수익율                  30.6654
ETF점수   

In [34]:
# df의 컬럼 추출
df.columns

Index(['외화증권영문명', '총보유계좌수', '총보유수량', '당사평균보유수량', '당사평균보유비중비율', '당사평균평가손익',
       '당사평균매입단가', '당사평균수익율', '당사평균보유기간일수', '분포상위10퍼센트수치', '분포상위30퍼센트수치',
       '분포상위50퍼센트수치', '분포상위70퍼센트수치', '분포상위90퍼센트수치', '기준종가', '손실투자자비율',
       '수익투자자비율', '신규매수계좌수', '전량매도계좌수', '종목조회건수', '관심종목등록건수', '종목종가',
       '전일대비증감율', '누적거래수량', '거래대금', '매도체결합계수량', '매수체결합계수량', '대상ETF티커',
       '1개월총수익율', '3개월총수익율', '1년총수익율', 'ETF점수', 'ETFZ점수', 'Z점수순위', '누적수익율Z점수',
       '정보비율Z점수', '샤프지수Z점수', '상관관계Z점수', '트래킹에러Z점수', '최대낙폭Z점수', '변동성Z점수'],
      dtype='object')

In [35]:
#df를 csv로 저장 단, 한글을 깨지지 않도록
df.to_csv('etf_df.csv', encoding='utf-8-sig')